In [ ]:
import cv2
import os
from glob import glob
import numpy as np
import random

In [ ]:
def save_frames(video_path, output_path, subject_name, action_name, filename, 
                clip_len, sample_len):
    
    if not os.path.exists(os.path.join(output_path, subject_name)):
        os.mkdir(os.path.join(output_path, subject_name))
        
    if not os.path.exists(os.path.join(output_path, subject_name, action_name)):
        os.mkdir(os.path.join(output_path, subject_name, action_name))
        
    cap = cv2.VideoCapture(os.path.join(video_path, filename))
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    frames = []
    # Modify Here
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    
    assert frame_count == len(frames)

    clipped_frames = [frames[i:i+clip_len] for i in range(0, frame_count, clip_len)]
  
    if clip_len == 150:
        sampled_frames = []
        for i in range(0, len(clipped_frames), 2):
            if len(clipped_frames[i]) == 150:
                sampled_frames.append(clipped_frames[i])
        clipped_frames = sampled_frames
        
    indicies = [i for i in range(0, clip_len, 5)]
    
    processed = []
    for f in clipped_frames: # f : [150 장]
        if len(f) == clip_len:
            processed.append([f[i] for i in indicies])

    for idx, clips in enumerate(processed): # clips [30 장]
        if not os.path.exists(os.path.join(output_path, subject_name, action_name, str(idx))):
            os.mkdir(os.path.join(output_path, subject_name, action_name, str(idx)))
        i = 1
        for frame in clips:
            if i >= 10:
                cv2.imwrite(filename=os.path.join(output_path, subject_name, action_name, str(idx), f'{i}.jpg'), img=frame)
            else:
                cv2.imwrite(filename=os.path.join(output_path, subject_name, action_name, str(idx), f'0{i}.jpg'), img=frame)
            i += 1
    # Release the VideoCapture
    cap.release()

In [ ]:
def process_video(video_path, output_path, clip_len, sample_len):
    
    # create processed data directory
    if not os.path.exists(output_path):
        os.mkdir(output_path)
        os.mkdir(os.path.join(output_path, 'train'))
        os.mkdir(os.path.join(output_path, 'val'))
        os.mkdir(os.path.join(output_path, 'test'))
        
    train_dir = os.path.join(output_path, 'train')
    val_dir = os.path.join(output_path, 'val')
    test_dir = os.path.join(output_path, 'test')
    
    videos = []
    timestamps = []
    subjects, actions = [], []
        
    for f in glob(os.path.join(video_path, '*.*')):    
        if f.split('.')[-1] == 'mp4':
            videos.append(f)
            subjects.append(f.split('.')[0].split('_')[-1])
            actions.append(f.split('_')[-2])
        else:
            timestamps.append(f)
            
    assert len(videos) == len(timestamps)

    data = list(zip(sorted(videos), sorted(timestamps)))
    subjects = np.unique(subjects)
    actions = np.unique(actions)
    
    # Select Validation, Test Subject
    random.seed(42)
    Vchoice, Tchoice = random.randint(0, len(subjects)), random.randint(0,len(subjects))
    
    Vsubject, Tsubject = subjects[Vchoice], subjects[Tchoice]
    
    # process video
    for video, timestamp in data:
        subject_name = video.split('.')[0].split('_')[-1]
        action_name = video.split('_')[-2]

        if subject_name == Vsubject:
            save_frames(video_path, val_dir, subject_name, action_name, 
                        video, clip_len, sample_len)
        elif subject_name == Tsubject:
            save_frames(video_path, test_dir, subject_name, action_name, 
                        video, clip_len, sample_len)
        else:
            save_frames(video_path, train_dir, subject_name, action_name, 
                        video, clip_len, sample_len)

In [ ]:
def check_process(video_path):
    # Frame by Frame
    video = cv.VideoCapture('/Users/moon/Downloads/video/2022-07-26 17-18-31_A2.mp4')
    # window name and size
    cv.namedWindow("video", cv.WINDOW_AUTOSIZE)
    count = 0
    while video.isOpened():
        # Read video capture
        ret, frame = video.read()

        if not ret:
            break

        count += 1
        # Display each frame
        cv.putText(frame, f'frame{count}', (10,100), cv.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 1, cv.LINE_AA)
        cv.imshow("video", frame)
        # show one frame at a time
        key = cv.waitKey(0)

        while key not in [ord('q'), ord('k')]:
            key = cv.waitKey(0)
        # Quit when 'q' is pressed
        if key == ord('q'):
            break

    # Release capture object
    video.release()
    # Exit and distroy all windows
    cv.destroyAllWindows()

In [ ]:
video_path = '/Users/moon/Downloads/video'
output_path = os.path.join(video_path, 'data')
clip_len = 150 # 150 or 300
sample_len = 30 # 30 or 60

In [ ]:
process_video(video_path, output_path, clip_len, sample_len)

저장 디렉토리

-- data (root)

    -- video
        -- 날짜
          -- subjects
            -- actions
  
    -- sensor
        -- 날짜
          -- subjects
            -- actions

Proccessed 디렉토리

-- video_path(root)

    -- data
      -- train
        -- subjects
          -- frame clips
      -- val
        -- subjects
          -- frame clips
      -- test
        -- subjects
          -- frame clips